# Cocoa Investigation

In [85]:
# import dependencies
import pandas as pd

In [86]:
# filepath to data
filename = '../Resources/crop_livestock_production.csv'
# read in table
area_data = pd.read_csv(filename)

In [87]:
area_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14020 entries, 0 to 14019
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       14020 non-null  object 
 1   Domain            14020 non-null  object 
 2   Area Code         14020 non-null  int64  
 3   Area              14020 non-null  object 
 4   Element Code      14020 non-null  int64  
 5   Element           14020 non-null  object 
 6   Item Code         14020 non-null  int64  
 7   Item              14020 non-null  object 
 8   Year Code         14020 non-null  int64  
 9   Year              14020 non-null  int64  
 10  Unit              14020 non-null  object 
 11  Value             13711 non-null  float64
 12  Flag              10539 non-null  object 
 13  Flag Description  14020 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 1.5+ MB


In [88]:
area_data.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,7,Angola,5312,Area harvested,661,"Cocoa, beans",1961,1961,ha,1600.0,F,FAO estimate
1,QCL,Crops and livestock products,7,Angola,5312,Area harvested,661,"Cocoa, beans",1962,1962,ha,1600.0,F,FAO estimate
2,QCL,Crops and livestock products,7,Angola,5312,Area harvested,661,"Cocoa, beans",1963,1963,ha,1400.0,F,FAO estimate
3,QCL,Crops and livestock products,7,Angola,5312,Area harvested,661,"Cocoa, beans",1964,1964,ha,1400.0,F,FAO estimate
4,QCL,Crops and livestock products,7,Angola,5312,Area harvested,661,"Cocoa, beans",1965,1965,ha,1400.0,F,FAO estimate


In [89]:
columns = area_data.columns
columns

Index(['Domain Code', 'Domain', 'Area Code', 'Area', 'Element Code', 'Element',
       'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value', 'Flag',
       'Flag Description'],
      dtype='object')

In [90]:
explore = area_data['Area Code'].unique()
explore

array([   7,   23,   53,   19,   21,   32,   37,  351,  214,   44,   45,
         46,   48,  107,   49,  250,   55,   56,   58,   60,   61,   66,
         74,   81,   86,   87,   89,   90,   91,   93,   95,  100,  101,
        109,  123,  129,  131,  135,  138,  145,  157,  159,  166,  168,
        170,  171,  189,  191,  244,  193,  197,   25,   38,  207,  216,
        176,  217,  219,  220,  226,  215,  155,  236, 5000, 5100, 5101,
       5102, 5105, 5200, 5204, 5206, 5207, 5300, 5302, 5303, 5304, 5500,
       5502, 5503, 5504, 5801, 5802, 5803, 5815, 5817], dtype=int64)

Columns to be dropped - Domain Code, Domain, Item Code, Item, Year Code, Flag, Flag Description, Area Code

Element Codes
5312 Area harvested (ha)
5419 Yield (hg/ha)
5510 Production (tonnes)

In [91]:
production = area_data.drop(['Domain Code', 'Domain', 'Area Code', 'Element Code',
       'Item Code', 'Item', 'Year Code', 'Flag',
       'Flag Description'], axis = 1)

In [92]:
production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14020 entries, 0 to 14019
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Area     14020 non-null  object 
 1   Element  14020 non-null  object 
 2   Year     14020 non-null  int64  
 3   Unit     14020 non-null  object 
 4   Value    13711 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 547.8+ KB


In [108]:
production.head()

,Area,Element,Year,Unit,Value
0,Angola,Area harvested,1961,ha,1600.0
1,Angola,Area harvested,1962,ha,1600.0
2,Angola,Area harvested,1963,ha,1400.0
3,Angola,Area harvested,1964,ha,1400.0
4,Angola,Area harvested,1965,ha,1400.0


In [111]:
production.columns

Index(['Area', 'Element', 'Year', 'Unit', 'Value'], dtype='object')

In [115]:
production.pivot(index=["Area", "Year"], columns = "Element", values="Value")

ValueError: Length of passed values is 14020, index implies 2.

In [95]:
area_harvested = production.loc[production['Element'] == "Area harvested"]

In [96]:
area_harvested = area_harvested.drop('Element', axis = 1)

In [97]:
area_harvested.columns = ['Country', 'Year', 'Unit', 'Area_harvested']

In [100]:
# repeat for other columns
prod = production.loc[production['Element'] == "Production"]
prod = prod.drop('Element', axis = 1)
prod.columns = ['Country', 'Year', 'Unit', 'Production']

p_yield = production.loc[production['Element'] == "Yield"]
p_yield = p_yield.drop('Element', axis = 1)
p_yield.columns = ['Country', 'Year', 'Unit', 'Production']

In [116]:
prod

,Country,Year,Unit,Production
118,Angola,1961,tonnes,400.0
119,Angola,1962,tonnes,400.0
120,Angola,1963,tonnes,300.0
121,Angola,1964,tonnes,300.0
122,Angola,1965,tonnes,400.0
...,...,...,...,...
14015,Net Food Importing Developing Countries,2015,tonnes,2124218.0
14016,Net Food Importing Developing Countries,2016,tonnes,2050952.0
14017,Net Food Importing Developing Countries,2017,tonnes,2475119.0
14018,Net Food Importing Developing Countries,2018,tonnes,2602619.0


In [121]:
# combine tables
df1 = pd.merge(area_harvested, prod, how = "outer", on=["Country", "Year"])
df2 = pd.merge(df1, p_yield, how = "outer", on=["Country", "Year"])
df2


,Country,Year,Unit_x,Area_harvested,Unit_y,Production_x,Unit,Production_y
0,Angola,1961,ha,1600.0,tonnes,400.0,hg/ha,2500.0
1,Angola,1962,ha,1600.0,tonnes,400.0,hg/ha,2500.0
2,Angola,1963,ha,1400.0,tonnes,300.0,hg/ha,2143.0
3,Angola,1964,ha,1400.0,tonnes,300.0,hg/ha,2143.0
4,Angola,1965,ha,1400.0,tonnes,400.0,hg/ha,2857.0
...,...,...,...,...,...,...,...,...
4750,Benin,1987,NaN,NaN,tonnes,70.0,NaN,NaN
4751,Benin,1988,NaN,NaN,tonnes,NaN,NaN,NaN
4752,Benin,1989,NaN,NaN,tonnes,NaN,NaN,NaN
4753,Benin,1990,NaN,NaN,tonnes,15.0,NaN,NaN


In [126]:
production.loc[(production["Area"]=="Benin")& (production["Year"]==1987)]

,Area,Element,Year,Unit,Value
408,Benin,Production,1987,tonnes,70.0


In [122]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4755 entries, 0 to 4754
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         4755 non-null   object 
 1   Year            4755 non-null   int64  
 2   Unit_x          4724 non-null   object 
 3   Area_harvested  4567 non-null   float64
 4   Unit_y          4739 non-null   object 
 5   Production_x    4587 non-null   float64
 6   Unit            4557 non-null   object 
 7   Production_y    4557 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 334.3+ KB


In [63]:
p_table = production.pivot(columns = 'Element', values = 'Value')

In [64]:
p_table

Element,Area harvested,Production,Yield
0,1600.0,NaN,NaN
1,1600.0,NaN,NaN
2,1400.0,NaN,NaN
3,1400.0,NaN,NaN
4,1400.0,NaN,NaN
...,...,...,...
14015,NaN,2124218.0,NaN
14016,NaN,2050952.0,NaN
14017,NaN,2475119.0,NaN
14018,NaN,2602619.0,NaN


In [55]:
stacked = production.stack()

In [57]:
unstacked = stacked.unstack()

In [58]:
unstacked

,Area,Element,Year,Unit,Value
0,Angola,Area harvested,1961,ha,1600
1,Angola,Area harvested,1962,ha,1600
2,Angola,Area harvested,1963,ha,1400
3,Angola,Area harvested,1964,ha,1400
4,Angola,Area harvested,1965,ha,1400
...,...,...,...,...,...
14015,Net Food Importing Developing Countries,Production,2015,tonnes,2.12422e+06
14016,Net Food Importing Developing Countries,Production,2016,tonnes,2.05095e+06
14017,Net Food Importing Developing Countries,Production,2017,tonnes,2.47512e+06
14018,Net Food Importing Developing Countries,Production,2018,tonnes,2.60262e+06


In [42]:
multi_prod = production.pivot(index=['Area', 'Year', 'Unit'], columns =['Element'], values=[ 'Value'])

ValueError: Shape of passed values is (14020, 2), indices imply (2, 2)